In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_pickle('temporary_tracks.pkl')
df = df[:200]
df = df[df['num_tracks_id'] >= 20]
# becuase we only converted the first 200:
df.head()



,pid,name,tracks,num_tracks,num_tracks_id
3,3,mat,"[4WJ7UMD4i6DOPzyXU5pZSz, 2K0HAp3di1AvmIjYYdPk4...",126,21
14,14,90's,"[0HDaKOlVAfUWXdFR2RhBtN, 0wvIGFIgbyz4JNwQhZgTv...",103,26
16,16,slow hands,"[2GOQVqZ3uVp7LKVAY1T0mk, 0Up7ojBDVmoYsqK8OShEe...",105,25
17,17,Mom's playlist,"[0PywzrV955BnXDC8FHAf0n, 5ke4Bsi02hoYUFZOWqvAQ...",79,22
28,28,Yeet,"[1B87ZDrD3Pno9hS4stamWO, 4nFksVP9B1uBG6XNvXwkz...",81,23


In [3]:
df.shape

(59, 5)

In [4]:
tracks_features = pd.read_pickle('tracks_features_processed.pkl')
tracks_features.set_index('id', inplace=True)
tracks_features = tracks_features[tracks_features['time_signature'] != 0]
tracks_features.head()

,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,,
7lmeHLHBe4nmXzuXc0HDjk,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0
1wsRitfRRtWyEapl0q22o8,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0
1hR0fIFK2qRG3f3RF70pb7,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0
2lbASgTSoDO7MTuLAXlTW0,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0
1MQTmpYOZ6fcMQc56Hdo7T,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0


In [5]:
tracks_features.shape

(1201216, 13)

In [6]:
from sklearn.cluster import DBSCAN
# executes dbscan on tracks. returns output of dbscan
def get_largest_cluster(playlist):
    X = []
    for track in playlist:
        X.append(list(tracks_features.loc[track]))
    return DBSCAN(eps=.01, min_samples=2).fit(X).labels_
# sklearn.cluster.DBSCAN(eps=0.5, *, min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)[source]¶
# def get_recommended_song(playlist):


In [9]:
# PCA'ed features
def absmax_scale(series):
    return (series - min(series)) / (series.max() - series.min())
for col in tracks_features.columns:
    tracks_features[col] = absmax_scale(tracks_features[col])
X = tracks_features.values
tracks_features.to_pickle('normalized_track_features.pkl')
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_new = pca.fit_transform(X)
print(pca.explained_variance_ratio_)
print(pca.get_feature_names_out(tracks_features.columns))

[0.28090916 0.23345019]
['pca0' 'pca1']


In [ ]:
print(pca.components_)
print(X[:4])

[[-0.13712252 -0.17026129 -0.48681924 -0.04841513 -0.1607562   0.10917618
  -0.04481526  0.65104034  0.40731321 -0.06469509 -0.26742111 -0.06906333
  -0.05361235]
 [ 0.01677309 -0.01997869 -0.02947698  0.17879979 -0.02061614 -0.96936529
   0.00555279  0.02602271  0.14182626 -0.01496623 -0.07193879 -0.01224592
  -0.00391374]]
[[0.00000000e+00 4.70000000e-01 9.78000000e-01 6.36363636e-01
  8.09050923e-01 1.00000000e+00 7.50257998e-02 2.62048193e-02
  1.09000000e-05 3.56000000e-01 5.03000000e-01 4.01469057e-01
  7.50000000e-01]
 [1.00000000e+00 5.99000000e-01 9.57000000e-01 1.00000000e+00
  8.03533911e-01 1.00000000e+00 1.94014448e-01 1.29518072e-02
  7.06000000e-05 1.55000000e-01 4.89000000e-01 3.36485227e-01
  7.50000000e-01]
 [0.00000000e+00 3.15000000e-01 9.70000000e-01 6.36363636e-01
  8.08673045e-01 1.00000000e+00 4.98452012e-01 2.34939759e-02
  2.03000000e-06 1.22000000e-01 3.70000000e-01 5.46926675e-01
  7.50000000e-01]
 [1.00000000e+00 4.40000000e-01 9.67000000e-01 1.00000000e+00

In [ ]:
import matplotlib.pyplot as plt
clusters = DBSCAN(eps=.5, min_samples=100).fit(X_new).labels_


In [ ]:
plt.scatter(X_new[np.where(clusters==0),0], X_new[np.where(clusters==0),1], color='blue')
plt.scatter(X_new[np.where(clusters==1),0], X_new[np.where(clusters==1),1], color='red')
plt.scatter(X_new[np.where(clusters==2),0], X_new[np.where(clusters==2),1], color='yellow')
plt.scatter(X_new[np.where(clusters==3),0], X_new[np.where(clusters==3),1], color='orange')
plt.scatter(X_new[np.where(clusters==4),0], X_new[np.where(clusters==4),1], color='green')
plt.show()

Error: Kernel is dead

In [ ]:
print(len(clusters))

Error: Kernel is dead